In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof

  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8489 sha256=732e190cab248e8d057e15e94d1e889aaf38dff3b740911ee536963e94cf3148
  Stored in directory: /root/.cache/pip/wheels/34/ba/79/c0260c6f1a03f420ec7673eff9981778f293b9107974679e36
Successfully built multitasking
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 46 kB 8.2 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
 'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["sugg"],axis=1)
  y = data["sugg"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(11,df.shape[0]-1):
        sugg = 0
        if df[each][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-10][3],
                df[each-9][3],
                df[each-8][3],
                df[each-7][3],
                df[each-6][3],
                df[each-5][3],
                df[each-4][3],
                df[each-3][3],
                df[each-2][3],
                df[each-1][3],
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object)
  scaled["sugg"] = last_column
  return scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df)
      df = row_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df      

In [3]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#df = pd.read_parquet("/content/drive/MyDrive/Colab Files/250_crypto_1d_scaled_yf.parquet")
from google.colab import drive
drive.mount("/content/gdrive")
symbols = get_crypto_syms()
dfs = []
for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 12:
          dfs.append(df)
data = process(dfs)
data.to_csv("1d_10pastcandle_newversion.csv")
xTrain, xTest, yTrain, yTest = spliting(data)
data

Mounted at /content/gdrive

1 Failed download:
- XNO-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-EUR: No data found for this date range, symb

,0,1,2,3,4,5,6,7,8,9,sugg
0,0.723113,0.000000,0.344140,0.098183,0.179437,1.000000,0.692987,0.409270,0.234882,0.115234,0.0
1,0.300546,0.541256,0.369221,0.426054,1.000000,0.785258,0.586811,0.464835,0.381146,0.000000,0.0
2,0.554290,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,1.0
3,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.0
4,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.135070,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1761617,0.000000,0.110425,0.174078,0.177459,0.408812,0.400589,0.305815,0.610451,0.722925,1.000000,1.0
1761618,0.000000,0.060724,0.063949,0.284657,0.276813,0.186399,0.477018,0.584317,0.848643,1.000000,1.0
1761619,0.000000,0.003369,0.233901,0.225708,0.131269,0.434825,0.546900,0.822992,0.981087,1.000000,0.0
1761620,0.000000,0.231311,0.223090,0.128333,0.432914,0.545369,0.822394,0.981023,1.000000,0.755756,0.0


In [12]:
model = Sequential()
adam = tf.keras.optimizers.Adam(
    learning_rate=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.add(Dense(4196, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (None, 4196)              46156     
                                                                 
 dense_44 (Dense)            (None, 2048)              8595456   
                                                                 
 dense_45 (Dense)            (None, 1024)              2098176   
                                                                 
 dense_46 (Dense)            (None, 100)               102500    
                                                                 
 dense_47 (Dense)            (None, 1)                 101       
                                                                 
Total params: 10,842,389
Trainable params: 10,842,389
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(xTrain,yTrain,epochs=200,batch_size=32768,validation_data=(xTest,yTest))
model.evaluate(xTest,yTest)
model.save(f"{model.evaluate(xTest,yTest)}.h5")

Epoch 1/222
 5/97 [>.............................] - ETA: 18:51 - loss: 5.1768 - accuracy: 0.4996

KeyboardInterrupt: 

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

In [ ]:
import os
files = os.listdir("/content/gdrive/MyDrive/Colab Files/")

data = pd.DataFrame()
for file in files:
  address = f"/content/gdrive/MyDrive/Colab Files/{file}"
  df = pd.read_csv(address)
  data = pd.concat([data,df])

In [ ]:
symbols = read_syms_from_txt()
len(symbols)
pieces = 15
new_arrays = np.array_split(symbols, pieces)
for i in new_arrays:
  dfs = []
  for x in i:
        data = yf.download(x,period="MAX",interval="1d",progress=False)
        if data.empty :
             print("Passing...")
        else:
               dfs.append(data)
  ndfs = dfs
  dfs = []
  for df in ndfs:
        if df.shape[0] > 12:
          dfs.append(df)
  data = process(dfs)
  data.to_csv(f"{x}.csv")         

In [3]:
address = "/datasets/dff/Colab Files/1d_10pastcandle_newversion.csv"
#from google.colab import drive
#drive.mount("/content/gdrive/")
data = pd.read_csv(address)
data = data.drop("Unnamed: 0",axis=1)
xTrain, xTest, yTrain, yTest = spliting(data)
data

(1585459, 10) (1585459,)
(176163, 10) (176163,)


,0,1,2,3,4,5,6,7,8,9,sugg
0,0.723113,0.000000,0.344140,0.098183,0.179437,1.000000,0.692987,0.409270,0.234882,0.115234,0.0
1,0.300546,0.541256,0.369221,0.426054,1.000000,0.785258,0.586811,0.464835,0.381146,0.000000,0.0
2,0.554290,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,1.0
3,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.0
4,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.135070,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1761617,0.000000,0.110425,0.174078,0.177459,0.408812,0.400589,0.305815,0.610451,0.722925,1.000000,1.0
1761618,0.000000,0.060724,0.063949,0.284657,0.276813,0.186399,0.477018,0.584317,0.848643,1.000000,1.0
1761619,0.000000,0.003369,0.233901,0.225708,0.131269,0.434825,0.546900,0.822992,0.981087,1.000000,0.0
1761620,0.000000,0.231311,0.223090,0.128333,0.432914,0.545369,0.822394,0.981023,1.000000,0.755756,0.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5f407dcd-dd82-4da3-94bb-bd4c89cde6ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>